## [Issues]
0. Data 가져오기
1. ID, Name and SSN (Not useful)
2. Age, Annual_Income, Num_of_Loan, Num_of_Delayed_Payment, Changed_Credit_Limit, Amount_invested_monthly, Outstanding_Debt Credit_Mix, Monthly_Balance Numerical but show as catogery (need to be fixed)
3. Occupation, CreditMix has value "__"
4. Data contains outliers
5. Num_Credit_Card has zeros
6. Type_of_Loan Need to rewrite as 8 columns
7. Num_Bank_Accounts contains negative values
8. Credit_History_Age,Payment_of_Min_Amount,Payment_Behaviour,'Credit_Mix' (needs Feature Engineering)
9. Target Columns is Imbalanced
10. A lot of missing data

# 0. 라이브러리

In [2]:
# Packages for EDA 
import matplotlib.pyplot as plt
import seaborn as sns 
import pandas as pd 
import numpy as np 

# Data Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer, MinMaxScaler, StandardScaler


from sklearn.metrics import mean_squared_error
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
import re 

# Modeling and evaluation 
# from sklearn.ensemble import (
#     BaggingClassifier,
#     ExtraTreesClassifier,
#     RandomForestClassifier,
#     StackingClassifier,
#     HistGradientBoostingClassifier
# )
# from xgboost import XGBClassifier
from sklearn.metrics import classification_report 
# import joblib # 파이프제작 파이프라인

# Packages options 
sns.set(rc={'figure.figsize': [14, 7]}, font_scale=1.2) # Standard figure size for all 
np.seterr(divide='ignore', invalid='ignore', over='ignore') ;

import warnings 
warnings.filterwarnings("ignore")

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject


# 1. CSV 가져오기

Colab에서

In [4]:
# from google.colab import drive # 드라이브에서 파일 가져오는 방식
# drive.mount("/content/drive")

# filename = "/content/drive/MyDrive/해커톤/Preprocessed_final_Data.csv"
# df = pd.read_csv(filename)
# df.head()

개인컴퓨터에서

In [5]:
df = pd.read_csv('./csv//Preprocessed_final_Data.csv', index_col=0)
# df.drop("Unnamed: 0", inplace=True, axis=1) # index_col은 0!
df.head(3)

,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,...,Occupation_Musician,Occupation_Scientist,Occupation_Teacher,Occupation_Writer,Payment_of_Min_Amount_Yes,Payment_Behaviour_High_spent_Medium_value_payments,Payment_Behaviour_High_spent_Small_value_payments,Payment_Behaviour_Low_spent_Large_value_payments,Payment_Behaviour_Low_spent_Medium_value_payments,Payment_Behaviour_Low_spent_Small_value_payments
0,23.0,19114.12,1824.843333,3.0,4.0,3.0,4.0,3.0,7.0,11.27,...,0,1,0,0,0,0,1,0,0,0
1,23.0,19114.12,1824.843333,3.0,4.0,3.0,4.0,-1.0,7.0,11.27,...,0,1,0,0,0,0,0,1,0,0
2,33.0,19114.12,1824.843333,3.0,4.0,3.0,4.0,3.0,7.0,6.27,...,0,1,0,0,0,0,0,0,1,0


In [6]:
# df.columns
target= 'Delay_from_due_date'
y = df[target] # 종속변인 추출
X = df.drop(target, axis=1) # 종속변인+중복컬럼 제외

# X = StandardScaler().fit_transform(X) <- 정규화는 나중에! 중요도평가할 때 음수값 있으면 안됌

In [7]:
# 1. VIF로 다중공선성 높은 변수순서로 제거 -> 망함
# X = df.drop(['Monthly_Inhand_Salary', 'Credit_Utilization_Ratio', 'Num_Credit_Card', 'Credit_Mix', 'Num_Bank_Accounts', 'Age', 'Monthly_Balance', 'Num_of_Delayed_Payment', 'Interest_Rate', 'Credit_History_Age', 'Num_Credit_Inquiries'], axis=1)

# 2 . 도메인적 feature 선정(필수컬럼)
# X = df[['Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit', 'Num_Credit_Inquiries', 'Outstanding_Debt', 'Credit_Score', 'Payment_of_Min_Amount_Yes', 'Credit-Builder Loan', 'Personal Loan', 'Debt Consolidation Loan', 'Student Loan', 'Payday Loan', 'Mortgage Loan', 'Auto Loan', 'Home Equity Loan']]
# 신용카드수, 신용카드이자율, 대출수, 대출유형, 평균 연체일, 체불횟수, 신용카드 한도 변경율, 신용조회수, 신용점수, 최소금 지불여부, 체불액

# 3. F-I로 컬럼 4개 지우기 + 도메인적 feautre선정 + VIF로 다중공선성 높은 변수 제거
selected_feature = ["Interest_Rate", "Num_Bank_Accounts", "Credit_History_Age", "Num_of_Delayed_Payment", "Outstanding_Debt", "Num_Credit_Inquiries", "Annual_Income", "Total_EMI_per_month", "Changed_Credit_Limit", "Num_of_Loan", "Amount_invested_monthly", "Credit_Score", "Payment_of_Min_Amount_Yes"]
X = df[selected_feature]

# 2. 클래스화, Data split, Resampling

y 클래스화
- 0일 이하 : 0
- 1일~5일 : 1
- 6일~30일 : 2
- 31일~90일 : 3

In [8]:
for i in range(len(y)):
    if y[i] <= 0 :
        y[i] = 0
    elif y[i] <= 5:
        y[i] = 1
    elif y[i] <= 30:
        y[i] = 2
    elif y[i] <= 90:
        y[i] = 3
    else:
        pass

In [9]:
y=y.astype("int")
y.value_counts()

2    73990
3    15106
1     9118
0     1786
Name: Delay_from_due_date, dtype: int64

train, test split

In [10]:
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42,stratify=y)

scaling(3개 중 랜덤포레스트 그리드서치 score 높은걸로 채택)

In [11]:
scalar = PowerTransformer(method='yeo-johnson', standardize=True).fit(X_train)
X_train = scalar.transform(X_train)

# scalar = MinMaxScaler()
# X_train = scalar.fit_transform(X_train)

# scalar = StandardScaler()
# X_train = scalar.fit_transform(X_train)

resampling

In [12]:
# 오버샘플링
from imblearn.over_sampling import SMOTE
rus = SMOTE(sampling_strategy='auto')
X_data_rus, y_data_rus = rus.fit_resample(X_train, y_train)

In [13]:
# # 언더샘플링
# from imblearn.under_sampling import RandomUnderSampler
# rus = RandomUnderSampler(random_state=156)
# X_data_rus, y_data_rus = rus.fit_resample(X, y)

In [14]:
y_data_rus.value_counts(normalize=True)

2    0.25
3    0.25
1    0.25
0    0.25
Name: Delay_from_due_date, dtype: float64

In [15]:
y.value_counts()

2    73990
3    15106
1     9118
0     1786
Name: Delay_from_due_date, dtype: int64

---
## Modeling

In [46]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
# from sklearn.neural_network import MLPClassifier
# from sklearn.svm import SVC
# from xgboost import XGBClassifier
# from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.calibration import CalibratedClassifierCV

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve

모델 4개로 성능 확인

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
#### 이중 분류
# #10개의 모델
# models = [
#     RandomForestClassifier(),
#     LogisticRegression(),
#     SVC(),
#     XGBClassifier(),
#     DecisionTreeClassifier(),
#     KNeighborsClassifier()
# ]


# rdict={'model':[],'acc_train':[], 'auc_train':[], 'acc_val':[],'precision':[],'recall':[],'f1_score':[], 'auc_val':[]}

    
# for clf in models:
#     clf = clf.fit(X_train, y_train)
# #1열:Test
#     y_hat = clf.predict(X_train)
#     results_train  = (round(accuracy_score(y_train,y_hat),2),round(roc_auc_score(y_train,y_hat),2))
# #2열:Val
#     y_hat = clf.predict(X_val)
#     results = (round(accuracy_score(y_val,y_hat),2),
#                     round(precision_score(y_val,y_hat),2),
#                     round(recall_score(y_val,y_hat),2),
#                     round(f1_score(y_val,y_hat),2),
#                     round(roc_auc_score(y_val,y_hat),2))

#     rdict['model'].append(clf); 
#     rdict['acc_train'].append(results_train[0]); rdict['auc_train'].append(results_train[1])
#     rdict['acc_val'].append(results[0])
#     rdict['precision'].append(results[1])
#     rdict['recall'].append(results[2])
#     rdict['f1_score'].append(results[3])
#     rdict['auc_val'].append(results[4])   

#     print(results_train, results)

# rdf = pd.DataFrame(data=rdict)
# rdf 

In [ ]:
#### 다중분류 버전 : y(0,1,2,3)를 위해 "clf.predict_proba"추가하여 score.
# models = [
#     # RandomForestClassifier(), # 미친듯이 잘 나옴
#     # MLPClassifier(),
#     # AdaBoostClassifier(), #성능낮음
#     # LogisticRegression(),
#     # SVC(), 너무오래걸림
#     # XGBClassifier(),
#     # SGDClassifier(loss = 'hinge'),
#     # CalibratedClassifierCV(method="sigmoid"),
#     # GradientBoostingClassifier(),
#     DecisionTreeClassifier(criterion= 'gini',
#                             max_depth= None,
#                             min_samples_leaf= 1,
#                             min_samples_split= 6),
#     KNeighborsClassifier(metric = 'manhattan', n_neighbors = 1, weights = 'uniform')
#     ]


# rdict={'model':[],'acc_train':[], 'auc_train':[], 'acc_test':[],'precision':[],'recall':[],'f1_score':[], 'auc_test':[]}

    
# for clf in models:
#     clf = clf.fit(X_train, y_train)
# #1열:Train
#     y_hat = clf.predict(X_train)
#     y_hat_pro = clf.predict_proba(X_train) # 멀티클래스의 경우 auc값을 구하려면 y예측의 확률값과 실제y값을 비교해줘야한다.
#     results_train  = (round(accuracy_score(y_train,y_hat),2),round(roc_auc_score(y_train,y_hat_pro, multi_class='ovr'),2))
# #2열:Test
#     y_hat = clf.predict(X_test)
#     y_hat_pro = clf.predict_proba(X_test)
#     results = (round(accuracy_score(y_test,y_hat),2),
#                     round(precision_score(y_test,y_hat, average="micro"),2),
#                     round(recall_score(y_test,y_hat,average="macro"),2),
#                     round(f1_score(y_test,y_hat,average="macro"),2),
#                     round(roc_auc_score(y_test,y_hat_pro, multi_class='ovr'),2))

#     rdict['model'].append(clf); 
#     rdict['acc_train'].append(results_train[0]); rdict['auc_train'].append(results_train[1])
#     rdict['acc_test'].append(results[0])
#     rdict['precision'].append(results[1])
#     rdict['recall'].append(results[2])
#     rdict['f1_score'].append(results[3])
#     rdict['auc_test'].append(results[4])   

#     print(results_train, results)

# rdf = pd.DataFrame(data=rdict)
# rdf

---
## gridsearchCV

### 1 ) Randomforest

In [16]:
from sklearn.ensemble import RandomForestClassifier
dt_model = RandomForestClassifier()
dt_model.fit(X_train, y_train)
# dt_model.predict(X_test)

RandomForestClassifier()

In [17]:
dt_model.get_params()
                            # criterion= 'gini',
                            # max_depth= None,
                            # min_samples_leaf= 1,
                            # min_samples_split= 6

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [18]:
param_dict = {
"n_estimators" : [100,200,300,400,500,800],
"max_features" :["auto","log2","sqrt"],
"max_depth" : [None, 4,6,8,10,15],
"min_samples_split" : [1,2,3,4,5,10],
"min_samples_leaf" : [1,2,3,4]
}


In [ ]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(dt_model,
                    param_grid = param_dict,
                    cv=5)
grid.fit(X_train, y_train)

In [ ]:
# GridSearchCV 결과 추출하여 DataFrame으로 변환
scores_df = pd.DataFrame(grid.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score', \
            'split0_test_score', 'split1_test_score', 'split2_test_score']]

In [ ]:
print("Best Parameters : ", grid.best_params_)
print("Best estimator : ", grid.best_estimator_)
print("Best Score : ", grid.best_score_)
print("Best Test Score : ", grid.score(X_test, y_test))

### 2 ) DecisionTree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train, y_train)
# dt_model.predict(X_test)

DecisionTreeClassifier()

In [ ]:
dt_model.get_params()
                            # criterion= 'gini',
                            # max_depth= None,
                            # min_samples_leaf= 1,
                            # min_samples_split= 6

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': None,
 'splitter': 'best'}

In [ ]:
param_dict = {
    "criterion":["gini", "entropy"],
    "max_depth":range(1,10),
    "min_samples_split":range(1,10),
    "min_samples_leaf":range(1,5)
}

In [ ]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(dt_model,
                    param_grid = param_dict,
                    cv=10,
                    verbose=1,
                    n_jobs=-1)
grid.fit(X_train, y_train)

Fitting 10 folds for each of 648 candidates, totalling 6480 fits


GridSearchCV(cv=10, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(1, 10),
                         'min_samples_leaf': range(1, 5),
                         'min_samples_split': range(1, 10)},
             verbose=1)

In [ ]:
# GridSearchCV 결과 추출하여 DataFrame으로 변환
scores_df = pd.DataFrame(grid.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score', \
            'split0_test_score', 'split1_test_score', 'split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'criterion': 'gini', 'max_depth': 1, 'min_sam...",NaN,648,NaN,NaN,NaN
1,"{'criterion': 'gini', 'max_depth': 1, 'min_sam...",0.490399,513,0.490153,0.489912,0.489405
2,"{'criterion': 'gini', 'max_depth': 1, 'min_sam...",0.490399,513,0.490153,0.489912,0.489405
3,"{'criterion': 'gini', 'max_depth': 1, 'min_sam...",0.490399,513,0.490153,0.489912,0.489405
4,"{'criterion': 'gini', 'max_depth': 1, 'min_sam...",0.490399,513,0.490153,0.489912,0.489405
...,...,...,...,...,...,...
643,"{'criterion': 'entropy', 'max_depth': 9, 'min_...",0.647549,94,0.646153,0.651366,0.645171
644,"{'criterion': 'entropy', 'max_depth': 9, 'min_...",0.647530,96,0.645960,0.651462,0.645026
645,"{'criterion': 'entropy', 'max_depth': 9, 'min_...",0.647568,91,0.646008,0.651511,0.645171
646,"{'criterion': 'entropy', 'max_depth': 9, 'min_...",0.647544,95,0.646057,0.651511,0.645026


In [ ]:
print("Best Parameters : ", grid.best_params_)
print("Best estimator : ", grid.best_estimator_)
print("Best Score : ", grid.best_score_)
print("Best Test Score : ", grid.score(X_test, y_test))

### 3 ) KNeibors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KN_model = KNeighborsClassifier()
KN_model.fit(X_train, y_train)
# dt_model.predict(X_test)

KNeighborsClassifier()

In [ ]:
KN_model.get_params()
# best param 'metric': 'manhattan', 'n_neighbors': 1, 'weights': 'uniform'

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [ ]:
param_dict = {
    'n_neighbors' : list(range(1,20)),
    'weights' : ["uniform", "distance"],
    'metric' : ['euclidean', 'manhattan', 'minkowski']
}

In [ ]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(KN_model,
                    param_grid = param_dict,
                    cv=10,
                    verbose=1,
                    n_jobs=-1)
grid.fit(X_train, y_train)

Fitting 10 folds for each of 114 candidates, totalling 1140 fits


GridSearchCV(cv=10, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'metric': ['euclidean', 'manhattan', 'minkowski'],
                         'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18, 19],
                         'weights': ['uniform', 'distance']},
             verbose=1)

In [ ]:
# GridSearchCV 결과 추출하여 DataFrame으로 변환
scores_df = pd.DataFrame(grid.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score', \
            'split0_test_score', 'split1_test_score', 'split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'metric': 'euclidean', 'n_neighbors': 1, 'wei...",0.921988,12,0.920842,0.924365,0.918714
1,"{'metric': 'euclidean', 'n_neighbors': 1, 'wei...",0.921988,12,0.920842,0.924365,0.918714
2,"{'metric': 'euclidean', 'n_neighbors': 2, 'wei...",0.890733,37,0.889372,0.891495,0.887146
3,"{'metric': 'euclidean', 'n_neighbors': 2, 'wei...",0.921973,16,0.920842,0.924317,0.918714
4,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei...",0.889889,42,0.890482,0.890096,0.888401
...,...,...,...,...,...,...
109,"{'metric': 'minkowski', 'n_neighbors': 17, 'we...",0.852301,80,0.850758,0.853412,0.850895
110,"{'metric': 'minkowski', 'n_neighbors': 18, 'we...",0.805263,111,0.802008,0.806497,0.806294
111,"{'metric': 'minkowski', 'n_neighbors': 18, 'we...",0.850279,83,0.848248,0.851144,0.849351
112,"{'metric': 'minkowski', 'n_neighbors': 19, 'we...",0.803820,113,0.800705,0.805049,0.805329


In [ ]:
print("Best Parameters : ", grid.best_params_)
print("Best estimator : ", grid.best_estimator_)
print("Best Score : ", grid.best_score_)
print("Best Test Score : ", grid.score(X_test, y_test))

Best Parameters :  {'metric': 'manhattan', 'n_neighbors': 1, 'weights': 'uniform'}
Best estimator :  KNeighborsClassifier(metric='manhattan', n_neighbors=1)
Best Score :  0.9512820284675827
Best Test Score :  0.9546672973825292
